In [1]:
# training dataset
training_data = [
    ['Yes', 'No','No','Yes','Some','$$$','No','Yes','French','0-10','Yes'],
    ['Yes', 'No', 'No', 'Yes', 'Full', '$', 'No', 'No', 'Thai', '30-60', 'No'],
    ['No', 'Yes', 'No', 'No', 'Some', '$', 'No', 'No', 'Burger', '0-10', 'Yes'],
    ['Yes', 'No', 'Yes', 'Yes', 'Full', '$', 'No', 'No', 'Thai', '10-30', 'Yes'],
    ['Yes', 'No', 'Yes', 'No', 'Full', '$$$', 'No', 'Yes', 'French', '>60', 'No'],
    ['No','Yes','No','Yes','Some','$$','Yes','Yes','Italian','0-10','Yes'],
    ['No', 'Yes', 'No', 'No', 'None', '$', 'Yes', 'No', 'Burger', '0-10', 'No'],
    ['No', 'No', 'No', 'Yes', 'Some', '$$', 'Yes', 'Yes', 'Thai', '0-10', 'Yes'],
    ['No', 'Yes', 'Yes', 'No', 'Full', '$', 'Yes', 'No', 'Burger', '>60', 'No'],
    ['Yes','Yes','Yes','Yes','Full','$$$','No','Yes','Italian','10-30','No'],
    ['No', 'No', 'No', 'No', 'None', '$', 'No', 'No', 'Thai', '0-10', 'No'],
    ['Yes','Yes','Yes','Yes','Full','$','No','No','Burger','30-60','Yes']
]

In [2]:
# column labels
header = ["Alternate", "Bar", "Fri/Sat", "Hungry", "Patrons", "Price", "Raining", "Reservation" , "Type", "WaitEstimate", "WillWait"]

In [3]:
def unique_vals(rows, col):
    """ This return a set of unique values in a particular column """
    return set([row[col] for row in rows])

unique_vals(training_data, 8)

{'Burger', 'French', 'Italian', 'Thai'}

In [4]:
def class_counts(rows):
    counts = {} # a dictionary of labels.
    for row in rows: # for every row in rows   
        label = row[-1] # assuming labels are the last column in data frame
        if label not in counts: # label is not in counts
            counts[label] = 0
        counts[label] += 1
    return counts

In [5]:
def is_numeric(value):
    """ Testing whether the values are numeric """
    return isinstance(value, int) or isinstance(value, float)

In [6]:
# # Checking categorical and numeric values.
# for i in range(len(training_data)):
#     print("Is", training_data[i][0], "numeric?" , is_numeric(training_data[i][0]))

In [7]:
class Question:
    
    def __init__(self, column, value):
        self.column = column
        self.value = value
    
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    
    def __repr__(self):
    
        """ Helper method to that the question can be printed in 
        a readable format."""
        
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s" % (
        header[self.column], condition, str(self.value))

In [8]:
q = Question(0, "Yes")
q

Is Alternate == Yes

In [9]:
example1 = training_data[0]
example1

['Yes', 'No', 'No', 'Yes', 'Some', '$$$', 'No', 'Yes', 'French', '0-10', 'Yes']

In [10]:
q.match(example1) # is the first example is green. this will be true

True

In [11]:
q1 = Question(2, "Yes")
example2 = training_data[0]
q1.match(example2) # is False because the string instance is 'Apple'.

False

In [12]:
def partition(rows, question):
    
    true_rows, false_rows = [], []
    
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [13]:
true_rows, false_rows = partition(training_data, Question(0, 'Yes'))
true_rows # get back rows were the first element is equivalen to 'Red'

[['Yes',
  'No',
  'No',
  'Yes',
  'Some',
  '$$$',
  'No',
  'Yes',
  'French',
  '0-10',
  'Yes'],
 ['Yes', 'No', 'No', 'Yes', 'Full', '$', 'No', 'No', 'Thai', '30-60', 'No'],
 ['Yes', 'No', 'Yes', 'Yes', 'Full', '$', 'No', 'No', 'Thai', '10-30', 'Yes'],
 ['Yes', 'No', 'Yes', 'No', 'Full', '$$$', 'No', 'Yes', 'French', '>60', 'No'],
 ['Yes',
  'Yes',
  'Yes',
  'Yes',
  'Full',
  '$$$',
  'No',
  'Yes',
  'Italian',
  '10-30',
  'No'],
 ['Yes',
  'Yes',
  'Yes',
  'Yes',
  'Full',
  '$',
  'No',
  'No',
  'Burger',
  '30-60',
  'Yes']]

In [14]:
def gini(rows):
    
    """ Gini Impurity is a measurement of the likelihood of 
        an incorrect classification of a new instance of a 
        random variable, if that new instance were randomly 
        classified according to the distribution of class 
        labels from the data set. Source: https://bambielli.com
        /til/2017-10-29-gini-impurity """
    
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts: # for labels in counts
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [15]:
def info_gain(left, right, current_uncertainty):
    
    """ Information gain is the reduction in entropy 
        or surprise by transforming a dataset and is
        often used in training decision trees. 
        Information gain is calculated by comparing
        the entropy of the dataset before and after 
        a transformation. """
    
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [16]:
current_uncertainty = gini(training_data)
current_uncertainty

0.5

In [17]:
true_rows, false_rows = partition(training_data, Question(3, 'Yes'))
info_gain(true_rows, false_rows, current_uncertainty)

0.12857142857142861

In [18]:
def find_best_split(rows):
    
    """ Find the best questions to ask by iterating over every
    feature/value and calculating the information gain. """
    
    best_gain = 0 # keeping track of best information gain
    best_question = None # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1 # number of columns
    
    for col in range(n_features): # for every feature
        
        values = set([row[col] for row in rows]) # value that is unique
        
        for val in values: # for every value
        
            question = Question(col, val)
            
            true_rows, false_rows = partition(rows, question) # attempt to split data set
            
            if len(true_rows) == 0 or len(false_rows) == 0: # if the data set is not divisible skip the split
                continue
            
            # information gain should be calculated after the split
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [19]:
# find the best question to ask our toy dataset
best_gain, best_question = find_best_split(training_data)
best_question

Is Patrons == Some

In [20]:
class Leaf:
    """
    A lead node is need to classify data.
        
    This will hold a dictionary of class ("Apple", "Grape", etc...) -> this will
    show the amount of times the class appears in the rows of the training dataset 
    when it gets to this leaf.
    """  
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [21]:
class Decision_Node:
    """
    This is a Decision Node and it asks a question.
    
    A reference to the question will be held here. In addition to the two child nodes
    that will branch off from this point.
    """
    def __init__(self, 
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [22]:
def build_tree(rows):
    """
    This builds our tree
    
    Rules for recursion: 
        1. Start from checking from the base case (this implies no further info. gain)
        2. There will be a big stack trace.
    """
    
    # attempt to partition the dataset on each of the attributes that are unique.
    # information gain needs to be calculated.
    # a lead is returned.
    gain, question = find_best_split(rows)
    
    # Base case:
    # There will be no more information gain.
    # Since no further questions are asked we will return a leaf
    if gain == 0:
        return Leaf(rows)
    
    # Assuming that we have reached this part of the code.
    # We are at a point where there is usefull features/values to split on.
    true_rows, false_rows = partition(rows, question)
    
    # Build the true branch using recursion
    true_branch = build_tree(true_rows)
    
    # Build the false branch using recursion
    false_branch = build_tree(false_rows)
    
    
    # The question node is returned
    # This is give you a record of the best feature/value to ask.
    # in addition, the branches to follow which depends on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [26]:
def fit(node, spacing=""):
    
    # base base: this is a way to say that we have reached a leaf
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return
    
    # The question at this node can be printed
    print(spacing + str(node.question))
    
    # This function needs to be called recursivly on the branch that is true
    print(spacing + "---> True: ")
    print_tree(node.true_branch, spacing + "  ")
    
    # The false branch needs to be called recursively as well
    print(spacing + "---> False: ")
    print_tree(node.false_branch, spacing + " ")

In [27]:
my_tree = build_tree(training_data)

In [28]:
fit(my_tree)

Is Patrons == Some
---> True: 
  Predict {'Yes': 4}
---> False: 
 Is Hungry == No
 ---> True: 
   Predict {'No': 4}
 ---> False: 
  Is Type == Italian
  ---> True: 
    Predict {'No': 1}
  ---> False: 
   Is Fri/Sat == No
   ---> True: 
     Predict {'No': 1}
   ---> False: 
    Predict {'Yes': 2}


In [35]:
def predict(row, node):
    """ See rules of recursion that is shown above. """
    
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions
        
    # A decision on whether to follow the true or flase branch
    if node.question.match(row):
        return predict(row, node.true_branch)
    else:
        return predict(row, node.false_branch)

In [36]:
def print_leaf(counts):
    
    """ A nicer way to print the predictions at a leaf. """
    
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total*100)) + "%"
    return probs

In [37]:
print_leaf(predict(training_data[0], my_tree))

{'Yes': '100%'}

In [38]:
print_leaf(predict(training_data[1], my_tree))

{'No': '100%'}

In [39]:
testing_data = [
    ['Yes', 'No','No','Yes','Some','$$$','No','Yes','French','0-10','Yes'],
    ['Yes', 'No', 'No', 'Yes', 'Full', '$', 'No', 'No', 'Thai', '30-60', 'No'],
    ['No', 'Yes', 'No', 'No', 'Some', '$', 'No', 'No', 'Burger', '0-10', 'Yes'],
    ['Yes', 'No', 'Yes', 'Yes', 'Full', '$', 'No', 'No', 'Thai', '10-30', 'Yes'],
]

In [41]:
for row in testing_data:
    print("Actual: %s Predicted: %s" % (row[-1], print_leaf(predict(row, my_tree))))

Actual: Yes Predicted: {'Yes': '100%'}
